# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

ModuleNotFoundError: No module named 'citipy'

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
city_lat = []
city_lng = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name.capitalize()
    
    
    # If the city is unique, then add it to a our cities list
    # Also add the latitude and longitude of that city to their
    # respective lists
    if city not in cities:
        cities.append(city)
        city_lat.append(lat_lng[0])
        city_lng.append(lat_lng[1])

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

In [ ]:
# First, make cities and lat_lngs into pd dataframe
weather_df = pd.DataFrame({
    "cities": cities, 
    "latitude": city_lat,
    "longitude": city_lng
})

# Add columns to store info 
# taken from responses to API

weather_df['temperature'] = ""
weather_df['humidity'] = ""
weather_df['cloudiness'] = ""
weather_df['wind speed'] = ""

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# Base url for finding cities' weather
base_url = "https://api.openweathermap.org/data/2.5/weather?"

# Create params dict that contains all 
# relevant parameters (or lat and lng)
# to pass to the request. Don't forget api key

params = {
    "appid": api_key
}

row_count = 0

print("Beginning Data Retrieval")
print("------------------------------")


for index, row in weather_df.iterrows():
    city = row["cities"]
    lat = row["latitude"]
    lon = row["longitude"]
    
    
    # Add latitude and longitude parameters
    # for current row
    params['q'] = city
    
    print(f'Processing Record {row_count}: {city}. ')
    row_count += 1
    
    city_weather = requests.get(base_url, params=params).json()

    
    # Append temp, humidity, cloudiness, and wind speed
    # to the dataframe using try/except
    
    try:
        temp = city_weather["main"]["temp"]
        humidity = city_weather["main"]["humidity"]
        cloudy = city_weather["clouds"]["all"]
        wind = city_weather["wind"]["speed"]
        
        weather_df.loc[index, "Latitude"] = city_weather["coord"]["lat"]
        weather_df.loc[index, "Longitude"] = city_weather["coord"]["lon"]
        weather_df.loc[index, "Temperature"] = temp
        weather_df.loc[index, "Humidity"] = humidity
        weather_df.loc[index, "Cloudiness"] = cloudy
        weather_df.loc[index, "Wind Speed"] = wind
        
    except:
        print("Error with data. Skipping...")
        
        weather_df.drop(labels=index, inplace=True)

In [ ]:
# Convert temperature to farhenheit
def k_to_f(temp):
    temp = (temp - 273.15) * 9/5 + 32
    return temp

weather_df["temperature"] = [k_to_f(temp) for temp in weather_df["temperature"]]

weather_df.head()

In [ ]:
weather_df.to_csv('weather_data.csv')

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
# Build a scatter plot for each weather type
plt.scatter(weather_df["latitude"], weather_df["temperature"], marker="o", c="pink", edgecolors="black")

# Incorporate the other graph properties
plt.title("Latitude vs. Temperature")
plt.ylabel("Max. Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save Figure
plt.savefig("./LatvsTemperature.png")

In [ ]:
# Show plot
plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
# Build a scatter plot for each weather type
plt.scatter(weather_df["latitude"], weather_df["humidity"], marker="o", c="pink", edgecolors="black")

# Incorporate the other graph properties
plt.title("Latitude vs. Humidity")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)

# Save Figure
plt.savefig("./LatvsHumidity.png")

In [ ]:
# Show plot
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
# Build a scatter plot for each weather type
plt.scatter(weather_df["latitude"], weather_df["cloudiness"], marker="o", c="pink", edgecolors="black")

# Incorporate the other graph properties
plt.title("Latitude vs. Cloudiness")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

# Save Figure
plt.savefig("./LatvsCloudiness.png")

In [ ]:
# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build a scatter plot for each weather type
plt.scatter(weather_df["latitude"], weather_df["wind speed"], marker="o", c="pink", edgecolors="black")

# Incorporate the other graph properties
plt.title("Latitude vs. Wind Speed")
plt.ylabel("wind speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)

# Save Figure
plt.savefig("./LatvsWind.png")

In [ ]:
# Show plot
plt.show()